In [ ]:
# default_exp backwards
# hide
from nbdev.showdoc import *
%load_ext autoreload
%autoreload 2

In [ ]:
from directed_graphs.datasets import DirectedStochasticBlockModelHelper
import torch

num_clusters = 7
num_nodes = 5*num_clusters

edge_index = np.ndarray([[0,1,0,3,4,3], [1,2,4,4,5,6]])
datasets = DirectedStochasticBlockModelHelper(num_nodes, num_clusters, edge_index, undir_prob=[0.8])
